# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="80" /> Procesamiento de Datos - Tarea 1
**Pontificia Universidad Católica de Chile**<br>
**Semestre 2023-1**<br>
**Profesor:** Juan Reutter. <br>
**Estudiantes:** Juan Bejarano T. y Jorge Rodriguez D.

---

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
    
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

### 1. Utiliza pyspark para cargar cada archivo como una "tabla", tal como lo hicimos en el tutorial. Recuerda que los archivos están disponibles en las instrucciones.

In [3]:
mascotas=sc.textFile("/content/mascotas-1-.csv")
personas=sc.textFile("/content/personas-1-.csv")

### 2. A partir de ahí, debes realizar las siguientes tareas:

#### Parte 1

Crea un programa que devuelve cada nombre de mascota y su tipo, junto con el
nombre de su dueño. Es decir, tu programa debe entregar, para cada mascota en tus datos, el nombre de la mascota, el tipo de mascota (por ejemplo, perro, gato, etc.) y el nombre del dueño de la mascota.

In [4]:
#Creamos una función para tomar los elementos necesarios de mascotas, teniendo como primer elemento el id del dueño y 
# como segundo elemento una tupla con el nombre y tipo de mascota
def fun1(line):
    row=line.split(";")
    return (int(row[3]),(row[1],row[2]))

#Creamos una función para tomar los elementos necesarios de personas, teniendo como primer elemento del id de persona y segundo elemento el nombre de la persona
def fun2(line):
    row=line.split(";")
    return (int(row[0]),row[1])

#Hacemos el join de las dos tablas después de operar las funciones anteriores 
final=mascotas.map(fun1).join(personas.map(fun2)).values().map(lambda data: (*data[0],data[1]))
final.take(7)

[('Debra', 'Conejo', 'Shane Arroyo'),
 ('Melvin', 'Tortuga', 'Shane Arroyo'),
 ('Michael', 'Pez', 'Shane Arroyo'),
 ('Jennifer', 'Pájaro', 'Shane Arroyo'),
 ('Christopher', 'Hámster', 'Shane Arroyo'),
 ('Cassandra', 'Tortuga', 'Shane Arroyo'),
 ('Elizabeth', 'Pez', 'Daniel Hardin')]

#### Parte 2
Crea un programa que devuelve, para cada número identificador y nombre de
persona, el número de mascotas que posee. En otras palabras, para cada persona en tus datos, tu programa debería mostrar el ID de la persona, su nombre, y el número total de mascotas que esa persona tiene.

In [5]:
#Usamos nuevamente fun2 para tomar los elementos necesarios de personas, teniendo como primer elemento del id de persona y segundo elemento el nombre de la persona
id_nombre=personas.map(fun2)

#Creamos una nueva tabla que contendrá cuantas veces aparece el id de cada dueño en mascotas
id_persona=mascotas.map(lambda line: line.split(";")[3]).map(lambda id: (int(id), 1)).reduceByKey(lambda a, b: a + b)

#Hacemos el join de las dos tablas
final2=id_nombre.join(id_persona).map(lambda data: (data[0],*data[1]))
final2.take(5)

[(4, 'Renee Scott MD', 7),
 (8, 'Evelyn Fitzgerald', 4),
 (12, 'Christopher Stewart', 2),
 (16, 'Pamela Christensen', 4),
 (20, 'Jeffrey Welch', 4)]